In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn

In [2]:
# -*- coding: utf-8 -*-
"""
Professional Crypto Trading Algorithm with CoinGecko Free API
Multi-asset support with advanced analytics
Free Tier - No API Key Required
Enhanced with robust rate limit handling
"""

# Core Financial Modules
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import os

# Visualization Enhancements
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# Performance Metrics
from sklearn.metrics import mean_squared_error

# API Integration
import requests

# DATA ACQUISITION MODULE WITH COINGECKO FREE API
class CryptoDataFetcher:
    def __init__(self, assets, start_date, end_date):
        self.tickers = assets
        self.start = start_date
        self.end = end_date
        self.datasets = {}
        self.base_url = "https://api.coingecko.com/api/v3"

        # CoinGecko coin IDs mapping
        self.coin_ids = {
            'BTC-USD': 'bitcoin',
            'ETH-USD': 'ethereum',
            'LTC-USD': 'litecoin',
            'XRP-USD': 'ripple',
            'ADA-USD': 'cardano',
            'DOGE-USD': 'dogecoin',
            'SOL-USD': 'solana',
            'BNB-USD': 'binancecoin',
            'DOT-USD': 'polkadot',
            'AVAX-USD': 'avalanche-2',
            'MATIC-USD': 'matic-network',
            'SHIB-USD': 'shiba-inu',
            'TRX-USD': 'tron',
            'LINK-USD': 'chainlink',
            'UNI-USD': 'uniswap',
            'ATOM-USD': 'cosmos',
            'ETC-USD': 'ethereum-classic',
            'XMR-USD': 'monero',
            'XLM-USD': 'stellar',
            'ALGO-USD': 'algorand',
            'VET-USD': 'vechain',
            'ICP-USD': 'internet-computer',
            'FIL-USD': 'filecoin',
            'FLOW-USD': 'flow',
            'EOS-USD': 'eos',
            'AAVE-USD': 'aave',
            'XTZ-USD': 'tezos',
            'NEO-USD': 'neo',
            'KLAY-USD': 'klay-token'
        }

        # API usage tracking
        self.api_calls = 0
        self.start_time = time.time()
        self.last_request_time = time.time()

    def fetch_market_data(self):
        """Retrieve multi-asset historical data with robust rate limit handling"""
        print(f"Starting data fetch for {len(self.tickers)} assets...")

        for i, symbol in enumerate(self.tickers):
            if symbol not in self.coin_ids:
                print(f"Unsupported symbol: {symbol}. Skipping...")
                self.datasets[symbol] = pd.DataFrame()
                continue

            coin_id = self.coin_ids[symbol]
            print(f"[{i+1}/{len(self.tickers)}] Fetching {symbol} ({coin_id})...")

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    # Rate limit management - free tier allows 50 calls/minute
                    current_time = time.time()
                    elapsed = current_time - self.last_request_time
                    if elapsed < 15:  # 24 requests/minute (safe margin)
                        sleep_time = 15 - elapsed
                        print(f"Rate control: Waiting {sleep_time:.1f}s")
                        time.sleep(sleep_time)

                    # Convert dates to timestamps
                    start_ts = int(datetime.strptime(self.start, '%Y-%m-%d').timestamp())
                    end_ts = int(datetime.strptime(self.end, '%Y-%m-%d').timestamp())

                    # API request configuration
                    endpoint = f"/coins/{coin_id}/market_chart/range"
                    url = f"{self.base_url}{endpoint}"
                    params = {
                        'vs_currency': 'usd',
                        'from': start_ts,
                        'to': end_ts
                    }

                    # Make request with timeout
                    response = requests.get(url, params=params, timeout=25)
                    self.api_calls += 1  # Track API usage
                    self.last_request_time = time.time()

                    # Handle API errors
                    if response.status_code == 429:
                        wait_time = (2 ** attempt) * 5  # Exponential backoff: 5s, 10s, 20s
                        print(f"Rate limited! Retry {attempt+1}/{max_retries} in {wait_time}s")
                        time.sleep(wait_time)
                        continue

                    if response.status_code != 200:
                        print(f"API error ({response.status_code}): {response.text[:100]}")
                        self.datasets[symbol] = pd.DataFrame()
                        break

                    data = response.json()

                    # Extract prices and create DataFrame
                    prices = data.get('prices', [])
                    if not prices:
                        print(f"No price data returned for {symbol}")
                        self.datasets[symbol] = pd.DataFrame()
                        break

                    df = pd.DataFrame(prices, columns=['timestamp', 'price'])
                    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
                    df.set_index('date', inplace=True)

                    # Resample to daily OHLC
                    ohlc = df['price'].resample('D').ohlc()
                    ohlc.columns = ['Open', 'High', 'Low', 'Close']

                    # Add volume if available
                    volumes = data.get('total_volumes', [])
                    if volumes:
                        volume_df = pd.DataFrame(volumes, columns=['timestamp', 'volume'])
                        volume_df['date'] = pd.to_datetime(volume_df['timestamp'], unit='ms')
                        volume_df.set_index('date', inplace=True)
                        daily_volumes = volume_df['volume'].resample('D').sum()
                        ohlc['Volume'] = daily_volumes

                    # Filter to exact date range
                    ohlc = ohlc.loc[self.start:self.end]

                    # Handle missing data
                    ohlc = ohlc.ffill().bfill().dropna()

                    if ohlc.empty:
                        print(f"No data returned for {symbol} in date range")
                        self.datasets[symbol] = pd.DataFrame()
                    else:
                        self.datasets[symbol] = ohlc
                        print(f"Successfully fetched {symbol} data ({len(ohlc)} records)")

                    # Break out of retry loop if successful
                    break

                except requests.exceptions.RequestException as e:
                    print(f"Request failed: {str(e)}")
                    if attempt < max_retries - 1:
                        wait_time = (2 ** attempt) * 3  # Shorter backoff for network issues
                        print(f"Retrying in {wait_time}s...")
                        time.sleep(wait_time)
                    else:
                        print("Max retries exceeded")
                        self.datasets[symbol] = pd.DataFrame()
                except Exception as e:
                    print(f"Error processing data: {str(e)}")
                    self.datasets[symbol] = pd.DataFrame()
                    break

        # Calculate API usage metrics
        elapsed = time.time() - self.start_time
        print(f"\nAPI Usage Report:")
        print(f"Total API Calls: {self.api_calls}")
        print(f"Time Elapsed: {elapsed:.2f} seconds")
        print(f"Requests per minute: {(self.api_calls / elapsed * 60):.2f}")

        return self.datasets


# TECHNICAL ANALYSIS ENGINE
class TradingSignalsGenerator:
    def __init__(self, price_data):
        self.data = price_data.copy()
        self.signals = pd.DataFrame(index=self.data.index)

    def compute_moving_averages(self, short_window=10, long_window=40):
        """Calculate dual moving averages"""
        self.data['Fast_MA'] = self.data['Close'].rolling(window=short_window, min_periods=1).mean()
        self.data['Slow_MA'] = self.data['Close'].rolling(window=long_window, min_periods=1).mean()
        return self.data

    def generate_trading_signals(self):
        """Create position-based trading signals"""
        self.signals['Position'] = np.where(self.data['Fast_MA'] > self.data['Slow_MA'], 1.0, 0.0)
        self.signals['Entry'] = self.signals['Position'].diff()
        return self.signals


# BACKTESTING FRAMEWORK
class PortfolioSimulator:
    def __init__(self, initial_capital=10000.0):
        self.initial_balance = initial_capital

    def simulate_strategy(self, price_data, signals):
        """Run trading strategy simulation"""
        portfolio = pd.DataFrame(index=signals.index)
        portfolio['Asset_Returns'] = price_data['Close'].pct_change().fillna(0)
        portfolio['Strategy_Returns'] = np.where(signals['Position'] == 1, portfolio['Asset_Returns'], 0)
        portfolio['Strategy_Value'] = (self.initial_balance * (1 + portfolio['Strategy_Returns']).cumprod())
        portfolio['Buy_Hold_Value'] = (self.initial_balance * (1 + portfolio['Asset_Returns']).cumprod())
        return portfolio


# VISUALIZATION MODULE - MODIFIED TO SAVE IMAGES
class TradingVisualizer:
    def __init__(self, output_dir="output_plots"):
        # Create output directory if it doesn't exist
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        print(f"Visualizations will be saved to: {os.path.abspath(output_dir)}")

    def plot_price_signals(self, asset_data, signals, symbol):
        """Save price action with trading signals as image"""
        fig, ax = plt.subplots(figsize=(14, 7), dpi=100)  # Reduced DPI for smaller files

        # Price and indicators
        ax.plot(asset_data['Close'], label='Closing Price', linewidth=1.5, color='navy')
        ax.plot(asset_data['Fast_MA'], label='Fast MA (10-day)', linewidth=1.2, color='orange')
        ax.plot(asset_data['Slow_MA'], label='Slow MA (40-day)', linewidth=1.2, color='purple')

        # Signal markers
        buy_signals = signals.loc[signals['Entry'] == 1.0]
        sell_signals = signals.loc[signals['Entry'] == -1.0]

        ax.plot(buy_signals.index, asset_data.loc[buy_signals.index, 'Close'],
                '^', markersize=10, color='green', alpha=0.9, label='Buy Signal')
        ax.plot(sell_signals.index, asset_data.loc[sell_signals.index, 'Close'],
                'v', markersize=10, color='red', alpha=0.9, label='Sell Signal')

        # Formatting
        ax.set_title(f'{symbol} Trading Signals', fontsize=16, fontweight='bold')
        ax.set_ylabel('Price (USD)', fontsize=12)
        ax.set_xlabel('Date', fontsize=12)
        ax.legend(loc='best', fontsize=10)
        ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
        plt.xticks(rotation=45, fontsize=9)
        plt.yticks(fontsize=9)
        ax.grid(True, alpha=0.2)
        plt.tight_layout()

        # Save instead of returning
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{self.output_dir}/{symbol}_signals_{timestamp}.png"
        plt.savefig(filename, bbox_inches='tight')
        plt.close(fig)  # Close figure to free memory
        print(f"Saved signal plot: {filename}")

    def plot_performance_comparison(self, portfolio, symbol):
        """Save strategy vs buy-and-hold comparison as image"""
        fig, ax = plt.subplots(figsize=(14, 7), dpi=100)  # Reduced DPI for smaller files

        # Plot portfolio values
        ax.plot(portfolio['Strategy_Value'], label='Trading Strategy', linewidth=2, color='blue')
        ax.plot(portfolio['Buy_Hold_Value'], label='Buy & Hold', linewidth=2, color='green')

        # Annotate final values
        final_strategy = portfolio['Strategy_Value'].iloc[-1]
        final_buyhold = portfolio['Buy_Hold_Value'].iloc[-1]

        # Strategy annotation
        ax.annotate(f'Strategy: ${final_strategy:,.2f}',
                    xy=(portfolio.index[-1], final_strategy),
                    xytext=(10, 10), textcoords='offset points',
                    fontsize=11, bbox=dict(boxstyle='round,pad=0.4', fc='lightblue', alpha=0.8),
                    arrowprops=dict(arrowstyle='->', color='black'))

        # Buy & Hold annotation
        ax.annotate(f'Buy & Hold: ${final_buyhold:,.2f}',
                    xy=(portfolio.index[-1], final_buyhold),
                    xytext=(10, -30), textcoords='offset points',
                    fontsize=11, bbox=dict(boxstyle='round,pad=0.4', fc='lightgreen', alpha=0.8),
                    arrowprops=dict(arrowstyle='->', color='black'))

        # Formatting
        ax.set_title(f'{symbol} Portfolio Value Comparison', fontsize=16, fontweight='bold')
        ax.set_ylabel('USD Value', fontsize=12)
        ax.set_xlabel('Date', fontsize=12)
        ax.legend(loc='best', fontsize=10)
        ax.grid(True, alpha=0.2)
        plt.xticks(rotation=45, fontsize=9)
        plt.yticks(fontsize=9)
        plt.tight_layout()

        # Save instead of returning
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{self.output_dir}/{symbol}_performance_{timestamp}.png"
        plt.savefig(filename, bbox_inches='tight')
        plt.close(fig)  # Close figure to free memory
        print(f"Saved performance plot: {filename}")


# PERFORMANCE ANALYTICS
def calculate_performance_metrics(portfolio):
    """Compute key performance indicators"""
    returns = portfolio['Strategy_Returns']
    total_return = (portfolio['Strategy_Value'].iloc[-1] / portfolio['Strategy_Value'].iloc[0]) - 1

    # Risk metrics
    volatility = returns.std() * np.sqrt(252) if not returns.empty else 0
    sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252) if returns.std() != 0 else 0

    # Drawdown calculation
    peak = portfolio['Strategy_Value'].cummax()
    drawdown = (portfolio['Strategy_Value'] - peak) / peak
    max_drawdown = drawdown.min() if not drawdown.empty else 0

    metrics = {
        'Total Return': f"{total_return:.2%}",
        'Annual Volatility': f"{volatility:.2%}",
        'Sharpe Ratio': f"{sharpe_ratio:.2f}",
        'Max Drawdown': f"{max_drawdown:.2%}",
        'MSE vs Buy&Hold': f"{mean_squared_error(portfolio['Strategy_Value'], portfolio['Buy_Hold_Value']):,.2f}"
    }
    return metrics


# EXECUTION PIPELINE
if __name__ == "__main__":
    # Configuration
    ASSETS = [
        'BTC-USD', 'ETH-USD', 'LTC-USD', 'XRP-USD', 'ADA-USD',
        'DOGE-USD', 'SOL-USD', 'BNB-USD', 'DOT-USD', 'AVAX-USD',
        'MATIC-USD', 'SHIB-USD', 'TRX-USD', 'LINK-USD', 'ATOM-USD'
    ]

    # Update the START date to be within the last 365 days from the END date
    END = datetime.now().strftime('%Y-%m-%d')  # Set end date to today
    START = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')  # 1 year back
    INITIAL_CAPITAL = 10000.0
    OUTPUT_DIR = "trading_visualizations"  # Directory for saving plots

    print("\n" + "="*70)
    print("FREE CRYPTO TRADING SYSTEM")
    print("Powered by CoinGecko Free API")
    print("Enhanced with Robust Rate Limit Handling")
    print("="*70)
    print(f"Assets: {', '.join(ASSETS)}")
    print(f"Date Range: {START} to {END}")
    print(f"Initial Capital: ${INITIAL_CAPITAL:,.2f}")
    print(f"Using CoinGecko Free Tier API (no key required)\n")

    # Initialize visualizer with output directory
    visualizer = TradingVisualizer(output_dir=OUTPUT_DIR)

    # Data pipeline
    print("Fetching market data from CoinGecko API...")
    start_time = time.time()
    data_engine = CryptoDataFetcher(ASSETS, START, END)
    market_datasets = data_engine.fetch_market_data()
    elapsed = time.time() - start_time
    print(f"\nData retrieval complete in {elapsed:.2f} seconds")

    # Process each asset
    for symbol, dataset in market_datasets.items():
        if dataset.empty:
            print(f"\nSkipping {symbol} due to empty dataset")
            continue

        print("\n" + "="*70)
        print(f"ANALYZING: {symbol}")
        print("="*70)
        print(f"Data range: {dataset.index[0].strftime('%Y-%m-%d')} to {dataset.index[-1].strftime('%Y-%m-%d')}")
        print(f"Records: {len(dataset)}")
        print(f"First Close: ${dataset['Close'].iloc[0]:,.2f}")
        print(f"Last Close: ${dataset['Close'].iloc[-1]:,.2f}")
        price_change = ((dataset['Close'].iloc[-1] / dataset['Close'].iloc[0] - 1) * 100)
        print(f"Price Change: {price_change:.2f}%")

        # Signal generation
        analysis_engine = TradingSignalsGenerator(dataset)
        data_with_indicators = analysis_engine.compute_moving_averages(short_window=10, long_window=40)
        signal_data = analysis_engine.generate_trading_signals()

        # Count signals
        buy_count = (signal_data['Entry'] == 1.0).sum()
        sell_count = (signal_data['Entry'] == -1.0).sum()
        print(f"\nStrategy Signals:")
        print(f"Buy Signals: {buy_count}, Sell Signals: {sell_count}")

        # Backtesting
        simulator = PortfolioSimulator(INITIAL_CAPITAL)
        portfolio_results = simulator.simulate_strategy(dataset, signal_data)

        # Calculate final values
        final_strategy = portfolio_results['Strategy_Value'].iloc[-1]
        final_buyhold = portfolio_results['Buy_Hold_Value'].iloc[-1]
        strategy_profit = final_strategy - INITIAL_CAPITAL
        buyhold_profit = final_buyhold - INITIAL_CAPITAL
        print(f"\nBacktest Results:")
        print(f"Final Strategy Value: ${final_strategy:,.2f} (Profit: ${strategy_profit:,.2f})")
        print(f"Final Buy & Hold Value: ${final_buyhold:,.2f} (Profit: ${buyhold_profit:,.2f})")

        # Performance analysis
        performance = calculate_performance_metrics(portfolio_results)
        print("\nPerformance Metrics:")
        for metric, value in performance.items():
            print(f"{metric}: {value}")

        # Visualization - now saves directly to files
        visualizer.plot_price_signals(data_with_indicators, signal_data, symbol)
        visualizer.plot_performance_comparison(portfolio_results, symbol)

    print("\n" + "="*70)
    print("ANALYSIS COMPLETE - VISUALIZATIONS SAVED TO FILES")
    print("="*70)
    # Removed plt.show() to prevent display


FREE CRYPTO TRADING SYSTEM
Powered by CoinGecko Free API
Enhanced with Robust Rate Limit Handling
Assets: BTC-USD, ETH-USD, LTC-USD, XRP-USD, ADA-USD, DOGE-USD, SOL-USD, BNB-USD, DOT-USD, AVAX-USD, MATIC-USD, SHIB-USD, TRX-USD, LINK-USD, ATOM-USD
Date Range: 2024-06-21 to 2025-06-21
Initial Capital: $10,000.00
Using CoinGecko Free Tier API (no key required)

Visualizations will be saved to: /content/trading_visualizations
Fetching market data from CoinGecko API...
Starting data fetch for 15 assets...
[1/15] Fetching BTC-USD (bitcoin)...
Rate control: Waiting 15.0s
Successfully fetched BTC-USD data (366 records)
[2/15] Fetching ETH-USD (ethereum)...
Rate control: Waiting 14.9s
Successfully fetched ETH-USD data (366 records)
[3/15] Fetching LTC-USD (litecoin)...
Rate control: Waiting 15.0s
Successfully fetched LTC-USD data (366 records)
[4/15] Fetching XRP-USD (ripple)...
Rate control: Waiting 15.0s
Successfully fetched XRP-USD data (366 records)
[5/15] Fetching ADA-USD (cardano)...
Rat